In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

def test():
    np.random.seed(42)
    tf.random.set_seed(42)

    # 生成数据
    num_samples = 1000
    user_ability = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)
    problem_difficulty = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)
    data = pd.DataFrame(
        {
            "user_ability": user_ability,
            "problem_difficulty": problem_difficulty,
            "accuracy": [
                (ua / 1000) * (1 - (pd / 1000)) > 0.5
                for ua, pd in zip(user_ability, problem_difficulty)
            ],
        }
    )

    X = data[["user_ability", "problem_difficulty"]]
    y = data["accuracy"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = Sequential(
        [
            Input(shape=(2,)),  # 使用Input层来定义输入形状
            Dense(64, activation="relu"),  # 第一层不再需要input_dim
            Dense(64, activation="relu"),  # 第二层密集层
            Dense(1, activation="sigmoid"),  # 输出层
        ]
    )

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    # 训练模型
    model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

    # 评估模型
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")


class BinaryClassifier:
    def __init__(self, input_dim, learning_rate=0.001, reg_lambda=0.001):
        self.model = self.build_model(input_dim, learning_rate, reg_lambda)

    def build_model(self, input_dim, learning_rate, reg_lambda):
        model = tf.keras.Sequential(
            [
                tf.keras.layers.Input(shape=(input_dim,)),
                tf.keras.layers.Dense(
                    128,
                    activation="relu",
                    kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                ),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(
                    64,
                    activation="relu",
                    kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                ),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation="sigmoid"),
            ]
        )
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
        )
        return model

    def train(self, X_train, y_train, epochs=100, batch_size=32, validation_data=None):
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,  # 没有进步的epochs数量，之后训练将被停止
            verbose=1,
            restore_best_weights=True  # 恢复最佳模型的权重
        )
        self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1,
            validation_data=validation_data,
            callbacks=[early_stopping]
        )

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        return loss, accuracy

    def predict(self, X):
        return self.model.predict(X)


def test2():
    np.random.seed(42)
    tf.random.set_seed(42)

    # 生成数据
    num_samples = 1000
    user_ability = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)
    problem_difficulty = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)

    data = pd.DataFrame(
        {
            "user_ability": user_ability,
            "problem_difficulty": problem_difficulty,
            "accuracy": [
                (ua / 1000) * (1 - (pd / 1000)) > 0.5
                for ua, pd in zip(user_ability, problem_difficulty)
            ],
        }
    )
    X = data[["user_ability", "problem_difficulty"]]
    y = data["accuracy"]

    # 数据标准化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 分割数据
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    # 初始化模型
    classifier = BinaryClassifier(input_dim=2)

    # 训练模型
    classifier.train(X_train, y_train)

    # 评估模型
    loss, accuracy = classifier.evaluate(X_test, y_test)
    print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    return classifier


class MLPClassifier:
    def __init__(
        self,
        input_shape=(2,),
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"],
    ):
        self.model = Sequential(
            [
                Input(shape=input_shape),
                Dense(64, activation="relu"),
                Dense(64, activation="relu"),
                Dense(1, activation="sigmoid"),
            ]
        )
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    def train(self, X_train, y_train, epochs=50, batch_size=10, verbose=1):
        self.model.fit(
            X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose
        )

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)

    def predict(self, X):
        return self.model.predict(X)


def usage():
    data = pd.DataFrame(
        {
            "user_ability": np.random.rand(100),
            "problem_difficulty": np.random.rand(100),
            "accuracy": np.random.randint(0, 2, size=100),
        }
    )

    X = data[["user_ability", "problem_difficulty"]]
    y = data["accuracy"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    classifier = MLPClassifier()
    classifier.train(X_train, y_train)
    loss, accuracy = classifier.evaluate(X_test, y_test)
    print(f"Test loss: {loss}, Test accuracy: {accuracy}")





In [3]:
test()

Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8144 - loss: 8.3242        
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9612 - loss: 0.4619   
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9649 - loss: 0.3750   
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9709 - loss: 0.3228   
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.9707 - loss: 0.3439 
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.9768 - loss: 0.3452 
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - accuracy: 0.9768 - loss: 0.3242 
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.9765 - loss: 0.3525 
Epoch 9/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.9765 - loss: 0.3540 
Epoch 10/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - accuracy: 0.9769 - loss: 0.3472 
Epoch 11/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - accuracy: 0.9731 - loss: 0.3433 
Epoch 12/50
80/80 ━━━━━━━━━━━━━━━━━━

In [7]:
test2()

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5232 - loss: 0.7792   
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.4665 
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.9777 - loss: 0.3104
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.9777 - loss: 0.2344
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9777 - loss: 0.1933
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.1801 
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.9777 - loss: 0.1516
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.1506 
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9785 - loss: 0.1363 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.9780 - loss: 0.1137
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9834 - loss: 0.1127
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 991us

In [5]:
usage()

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6372 - loss: 0.6915  
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6820 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6782 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6760 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6056 - loss: 0.6747 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6740 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6736 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6056 - loss: 0.6733 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6731 
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6056 - loss: 0.6729 
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6056 - loss: 0.6728 
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6056 - loss: 0.6726 


In [11]:
import numpy as np

# 假设 classifier 是我们已经训练好的模型
# 假设这些数据已经是标准化处理后的数据
new_data = np.array([
    [0.5, 0.3],   # 学生 1
    [-1.0, 1.5],  # 学生 2
    [0.2, -0.1]   # 学生 3
])
classifier = test2()
# 使用模型进行预测
predictions = classifier.predict(new_data)
print(predictions)

# 输出预测结果
for i, prediction in enumerate(predictions, 1):
    result = "能" if prediction == 1 else "不能"
    print(f"学生 {i} {result} 正确解答这道题目。")

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7607 - loss: 0.6855   
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.3593 
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.2504 
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9777 - loss: 0.2087 
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.9777 - loss: 0.1971
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.9777 - loss: 0.1539
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.9777 - loss: 0.1496
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.9788 - loss: 0.1376
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9730 - loss: 0.1234 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.9805 - loss: 0.1111
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - accuracy: 0.9827 - loss: 0.1066
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8

In [12]:
predictions

array([[1.8363704e-05],
       [8.8176808e-09],
       [2.3649289e-05]], dtype=float32)

In [13]:
import tensorflow as tf
import numpy as np

# 设置随机种子以确保复现性
np.random.seed(42)

# 生成随机数据
# 假设我们有1000个样本
student_abilities = np.random.randint(100, 5001, size=1000)
problem_difficulties = np.random.randint(100, 5001, size=1000)

# 将数据组合成一个特征矩阵
features = np.column_stack((student_abilities, problem_difficulties))

# 生成随机标签（例如，1表示解答正确，0表示解答错误）
labels = np.random.randint(0, 2, size=1000)

# 分割数据为训练集和测试集
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [14]:
# 创建一个简单的神经网络模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(2,)),  # 隐藏层，2个输入特征
    tf.keras.layers.Dense(1, activation='sigmoid')  # 输出层，使用sigmoid激活函数进行二分类
])

# 编译模型
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10


c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5535 - loss: 170.7511   
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5533 - loss: 94.5844  
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.4563 - loss: 45.8973
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.4967 - loss: 34.7728
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4818 - loss: 26.0629 
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4754 - loss: 19.7057 
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.4962 - loss: 15.3640
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.4895 - loss: 12.8097
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.4987 - loss: 11.4046
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4964 - loss: 10.5806 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.5358 - loss: 11.2614
Test Accuracy: 55.50%


In [22]:
def test3():
    np.random.seed(42)
    tf.random.set_seed(42)

    # 生成数据
    num_samples = 1000
    user_ability = np.random.randint(100, 5001, size=1000)
    problem_difficulty = np.random.randint(100, 5001, size=1000)

    data = pd.DataFrame(
        {
            "user_ability": user_ability,
            "problem_difficulty": problem_difficulty,
            "accuracy": [
                (ua / 1000) * (1 - (pd / 1000)) > 0.5
                for ua, pd in zip(user_ability, problem_difficulty)
            ],
        }
    )
    X = data[["user_ability", "problem_difficulty"]]
    y = data["accuracy"]

    # 数据标准化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 分割数据
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    # 初始化模型
    classifier = BinaryClassifier(input_dim=2)

    # 训练模型
    classifier.train(X_train, y_train)

    # 评估模型
    loss, accuracy = classifier.evaluate(X_test, y_test)
    print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    return classifier

In [23]:
test3()

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3812 - loss: 0.8490   
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8962 - loss: 0.5853 
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8931 - loss: 0.4333 
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9073 - loss: 0.3358 
Epoch 5/100
 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9688 - loss: 0.2254

c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9167 - loss: 0.2937 
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9316 - loss: 0.2615 
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9303 - loss: 0.2338 
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9611 - loss: 0.2094 
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9405 - loss: 0.2042 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9545 - loss: 0.1794 
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9557 - loss: 0.1677
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9497 - loss: 0.1709
Epoch 13/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9594 - loss: 0.1678 
Epoch 14/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9584 - loss: 0.1522 
Epoch 15/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9747 - loss: 0.1369 
Epoch 16/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

In [24]:
classfier = test3()

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8407 - loss: 0.6276   
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8909 - loss: 0.4148 
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.8925 - loss: 0.3264
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9114 - loss: 0.2733 
Epoch 5/100
 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9688 - loss: 0.1914

c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9101 - loss: 0.2582 
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9135 - loss: 0.2439 
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9343 - loss: 0.2184 
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.9470 - loss: 0.2036
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9417 - loss: 0.1946 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9540 - loss: 0.1774 
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9614 - loss: 0.1652 
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9625 - loss: 0.1541 
Epoch 13/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9645 - loss: 0.1491 
Epoch 14/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9546 - loss: 0.1541 
Epoch 15/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9672 - loss: 0.1424 
Epoch 16/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

In [25]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 假设这是您的原始数据
data = np.array([
    [200, 500],  # 学生 1 的能力和题目难度
    [3000, 4500],  # 学生 2 的能力和题目难度
    [1500, 4000],  # 学生 3 的能力和题目难度
])

# 创建一个MinMaxScaler对象
predictions = classfier.predict(data)

# 假设classifier是您已经训练好的模型
# 这里仅用作示例，实际模型应该已经训练并准备好了
# predictions = classifier.predict(scaled_data)

# 打印标准化后的数据
print("标准化后的数据:")
print(predictions)

# 假设这是预测结果，实际上应从您的模型中获得
# print("

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
标准化后的数据:
[[0.]
 [0.]
 [0.]]


In [28]:
np.random.seed(42)  # 为了可复现性
num_samples = 1000
X = np.random.randn(num_samples, 2) * 10
y = (X[:, 0] - X[:, 1] > 0).astype(int)

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 初始化并训练模型
classifier = BinaryClassifier(input_dim=2)
classifier.train(X_train_scaled, y_train, epochs=10)  # 减少epochs以快速看到结果

# 评估模型
loss, accuracy = classifier.evaluate(X_test_scaled, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

# 进行预测（可选）
predictions = classifier.predict(X_test_scaled[:5])
print("Sample predictions:", predictions)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 911us/step - accuracy: 0.7595 - loss: 0.6938 
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.9365 - loss: 0.4589
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9629 - loss: 0.2985
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9635 - loss: 0.2185 
Epoch 5/10
 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.1673

c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9809 - loss: 0.1662 
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9662 - loss: 0.1643 
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9841 - loss: 0.1287 
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9861 - loss: 0.1291 
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9764 - loss: 0.1313 
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9850 - loss: 0.1133 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9892 - loss: 0.1050 
Test loss: 0.09534616023302078, Test accuracy: 0.9950000047683716
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Sample predictions: [[9.9889934e-01]
 [1.3877620e-04]
 [9.9868226e-01]
 [1.0207914e-02]
 [9.9999827e-01]]


In [27]:
classifier = BinaryClassifier(input_dim=2)
classifier.train(X_train_scaled, y_train, epochs=10)  # 减少epochs以快速看到结果

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - accuracy: 0.6650 - loss: 0.7291 
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9534 - loss: 0.4911 
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9593 - loss: 0.3153 
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.9670 - loss: 0.2272
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.9815 - loss: 0.1770


c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9777 - loss: 0.1522 
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9799 - loss: 0.1395 
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9877 - loss: 0.1223
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9770 - loss: 0.1307
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9820 - loss: 0.1224 


In [29]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class RegressionModel:
    def __init__(self, input_dim, learning_rate=0.001):
        self.model = self.build_model(input_dim, learning_rate)

    def build_model(self, input_dim, learning_rate):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu", input_shape=(input_dim,)),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1, activation="sigmoid")  # 使用sigmoid确保输出在0到1之间
        ])
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])  # 使用均方误差作为损失函数
        return model

    def train(self, X_train, y_train, epochs=100, batch_size=32):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)

    def predict(self, X):
        return self.model.predict(X)

In [30]:
np.random.seed(42)
num_samples = 1000
abilities = np.random.uniform(100, 5000, num_samples)
difficulties = np.random.uniform(100, 5000, num_samples)
X = np.column_stack((abilities, difficulties))
y = abilities / (difficulties + np.random.normal(0, 100, num_samples))  # 添加一些噪声

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 初始化并训练模型
model = RegressionModel(input_dim=2)
model.train(X_train, y_train, epochs=50)  # 根据需要调整epochs

# 评估模型
mse, mae = model.evaluate(X_test, y_test)
print(f"Mean Squared Error: {mse}, Mean Absolute Error: {mae}")

# 进行预测（可选）
predictions = model.predict(X_test[:5])
print("Sample predictions:", predictions)

Epoch 1/50


c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 62.3479 - mae: 1.8600  
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61.9729 - mae: 1.7129
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - loss: 61.8006 - mae: 1.6206
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61.7652 - mae: 1.5944
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - loss: 61.7553 - mae: 1.5857
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 61.7506 - mae: 1.5799
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61.7478 - mae: 1.5750
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - loss: 61.7457 - mae: 1.5700
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - loss: 61.7440 - mae: 1.5654
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61.7428 - mae: 1.5613
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 61.7418 - mae: 1.5581
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 61.7411 - mae: 1.5556
Epoch 13/50
25/25 ━━━━━━━━━━━━━━━━

In [31]:
predictions

array([[0.4126028 ],
       [0.9999992 ],
       [0.5715437 ],
       [0.62355065],
       [1.        ]], dtype=float32)

In [32]:
X_test[:5]

array([[-0.37455127,  1.4635434 ],
       [ 1.11764609, -1.06968487],
       [-0.09444381,  1.09750877],
       [-0.45946522,  0.32042125],
       [ 1.57472323, -1.35287151]])

In [33]:
X_scaled[:5]

array([[-0.39630103, -1.10217857],
       [ 1.57695733,  0.11944663],
       [ 0.82789256,  1.25299211],
       [ 0.37125061,  0.77114327],
       [-1.14468466,  1.02568136]])

In [35]:
X[:5]

array([[1935.24658235, 1007.15135131],
       [4758.50010141, 2755.31464215],
       [3686.77031488, 4377.43459579],
       [3033.42657257, 3687.90194341],
       [ 864.49133817, 4052.14962452]])